In [ ]:
from google.colab import drive
import json
import glob

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


모델 파인튜닝

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 필요한 라이브러리 임포트
import json
from datasets import Dataset
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model

# 모델 경로 설정 (Google Drive 내부 경로로 변경)
model_path = "/content/drive/MyDrive/cv project/llama-3.2-Korean-Bllossom-3B"

# GPU 디바이스 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,  # GPU에서 효율적 연산
    device_map=None  # 직접 디바이스 지정
)
model = model.to(device)

# **여기에 pad_token 설정 추가**
tokenizer.pad_token = tokenizer.eos_token

# JSON 파일 로드 함수
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# JSON 파일 저장 함수
def save_json(file_path, data):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# 필수 키 확인 및 PROMPT 생성 함수
def create_prompt_and_instruction(json_data):
    # 기본 프롬프트 정의
    category_prompts = {
        "차트": (
            '너는 차트를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다.'
        ),
        "표": (
            '너는 표를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다.'
        ),
    }

    # 유형 키에 '표'가 포함되어 있는지 확인
    if "표" in json_data["유형"]:
        prompt = category_prompts["표"]
    else:
        prompt = category_prompts["차트"]

    # Instruction 생성
    instruction = (
        f"내용: {json_data['내용']}\n"
        "위 내용을 기반으로 요약문을 작성해줘."
    )
    return prompt, instruction

# 학습 데이터 로드
json_file_path = "/content/drive/MyDrive/cv project/기본표/학습데이터/result_table_with_summary.json"
json_file = load_json(json_file_path)

# 데이터 준비
data_list = []

for index, json_data in enumerate(json_file):
    # 필수 키 확인
    required_keys = ["data_id", "제목", "유형", "내용", "요약"]
    if not all(key in json_data for key in required_keys):
        print(f"[Error] JSON 항목 {index}가 필수 키 {required_keys}를 포함하지 않습니다.")
        continue

    # PROMPT 및 instruction 생성
    PROMPT, instruction = create_prompt_and_instruction(json_data)

    # 입력 텍스트와 대상 텍스트 준비
    input_text = f"{PROMPT}\n{instruction}\n"
    target_text = json_data["요약"]

    # 데이터 리스트에 추가
    data_list.append({
        'input_text': input_text,
        'target_text': target_text
    })

# Dataset 생성
from datasets import Dataset

dataset = Dataset.from_list(data_list)

# 데이터 전처리 함수 정의
def preprocess_function(examples):
    input_texts = examples['input_text']
    target_texts = examples['target_text']
    full_texts = [input_texts[i] + target_texts[i] for i in range(len(input_texts))]

    # 토크나이저 설정: padding=True로 변경
    tokenized_inputs = tokenizer(
        input_texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    tokenized_full = tokenizer(
        full_texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    labels = tokenized_full["input_ids"].clone()

    # 입력 부분은 -100으로 마스킹
    for i in range(len(input_texts)):
        input_len = tokenized_inputs['input_ids'].shape[1]
        labels[i, :input_len] = -100

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# 데이터셋 전처리
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names
)

# LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/cv project/finetuningllama",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=1e-4,
    fp16=True,
    save_total_limit=1,
    logging_steps=10,
    save_steps=200,
    evaluation_strategy="no"
)

# DataCollatorWithPadding 사용
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer, padding=True)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# 모델 학습
trainer.train()

# 파인튜닝된 모델 저장
trainer.save_model("/content/drive/MyDrive/cv project/finetuningllama")

print("파인튜닝된 모델이 '/content/drive/MyDrive/cv project/finetuningllama'에 저장되었습니다.")


Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/11294 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,0.000000
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,0.000000


Step,Training Loss
10,0.000000
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,0.000000


파인튜닝된 모델이 '/content/drive/MyDrive/cv project/finetuningllama'에 저장되었습니다.


파인튜닝 o , json 수정 x

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 모델 경로 설정
original_model_path = "/content/drive/MyDrive/cv project/llama-3.2-Korean-Bllossom-3B"
finetuned_model_path = "/content/drive/MyDrive/cv project/finetuningllama"

# GPU 디바이스 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 토크나이저를 원본 모델 경로에서 로드
tokenizer = AutoTokenizer.from_pretrained(original_model_path)
tokenizer.pad_token = tokenizer.eos_token  # pad_token 설정

# 모델을 원본 모델 경로에서 로드
model = AutoModelForCausalLM.from_pretrained(
    original_model_path,
    torch_dtype=torch.bfloat16,  # GPU에서 효율적 연산
    device_map=None  # 직접 디바이스 지정
)
model = model.to(device)
model.config.pad_token_id = tokenizer.pad_token_id

# 파인튜닝된 모델 로드 (LoRA 가중치 적용)
model = PeftModel.from_pretrained(model, finetuned_model_path)

# JSON 파일 로드 함수
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# JSON 파일 저장 함수
def save_json(file_path, data):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# Google Drive 내 JSON 파일 경로
json_file_path = "/content/drive/MyDrive/cv project/기본표/result_figure.json"  # 입력 JSON 파일 경로
output_file_path = "/content/drive/MyDrive/cv project/기본표/result_figure_with_summary.json"  # 저장 경로

# JSON 데이터 로드
json_file = load_json(json_file_path)

# 필수 키 확인 및 PROMPT 생성 함수 (이전 코드와 동일)
def create_prompt_and_instruction(json_data):
    # 기본 프롬프트 정의
    category_prompts = {
        "차트": (
            '너는 차트를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다. 아래의 차트 정보를 한 문장으로 정리해서 매우 자세하게 요약문을 생성하라.'
            '데이터에서 주요 패턴, 추세, 공통된 경향이 있다면 이를 포함하라.'
            '데이터에서 눈에 띄는 이상치나 중요한 점이 있다면 이를 포함하라.'
            '입력 데이터 외의 정보를 추가로 추측하지 말아라.'
        ),
        "표": (
            '너는 표를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다. 아래의 표 정보를 정리해서 하나의 문장으로 요약문을 생성하라.'
            '표의 데이터를 요약하며, 주요 항목 간의 비교 및 공통점 혹은 차이점을 강조하고 데이터에서 가장 두드러지는 부분을 서술하라.'
            '수치형 표의 경우 추세, 최댓값, 최솟값 등이 확인된다면 이와 비교 결과를 포함하라.'
            '표의 내용에서 벗어난 이야기를 생성하지 말아라.'
        ),
    }

    # 유형 키에 '표'가 포함되어 있는지 확인
    if "표" in json_data["유형"]:
        prompt = category_prompts["표"]
    else:
        prompt = category_prompts["차트"]

    # Instruction 생성
    instruction = (
        f"다음은 {json_data['제목']}에 대한 설명입니다.\n"
        f"유형: {json_data['유형']}\n"
        f"내용: {json_data['내용']}\n"
        "위 내용을 기반으로 요약문을 작성해줘."
    )
    return prompt, instruction

# JSON 파일의 모든 항목에 대해 실행
for index, json_data in enumerate(json_file):
    # 필수 키 확인
    required_keys = ["data_id", "제목", "유형", "내용"]
    if not all(key in json_data for key in required_keys):
        print(f"[Error] JSON 항목 {index}가 필수 키 {required_keys}를 포함하지 않습니다.")
        continue

    # PROMPT 및 instruction 생성
    PROMPT, instruction = create_prompt_and_instruction(json_data)

    # 입력 텍스트 생성
    input_text = f"{PROMPT}\n{instruction}\n"

    # 토크나이즈 및 텐서 변환
    input_ids = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    # 텍스트 생성
    outputs = model.generate(
        input_ids=input_ids['input_ids'],
        max_new_tokens=512,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_p=0.9
    )

    # 결과 텍스트 생성
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 입력 텍스트 부분을 제거하여 생성된 요약만 추출
    generated_summary = generated_text[len(input_text):].strip()

    print(f"[Result for JSON {index}]")
    print(generated_summary)
    print("\n" + "="*80 + "\n")

    # JSON 데이터에 요약 추가
    json_data["요약"] = generated_summary

# 업데이트된 JSON 파일 저장
save_json(output_file_path, json_file)

print(f"업데이트된 JSON 파일이 {output_file_path}에 저장되었습니다.")


Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[Result for JSON 0]
위의 차트는 학교 유형별 보건의료인력 수를 나타내며, 각 학교 유형별로 남자와 여자 인력의 비율을 보여준다. 고등학교는 남자 인력 비율이 600%, 여자 인력 비율이 2400%로 나타나며, 중학교는 남자 인력 비율이 450%, 여자 인력 비율이 1800%로 나타난다. 초등학교는 남자 인력 비율이 300%, 여자 인력 비율이 1200%로 나타난다. 이 차트는 학교 유형별 인력 비율의 변화를 보여주며, 각 학교 유형별로 인력 비율이 상당히 다를 수 있음을 시사한다. 또한, 각 학교 유형별로 인력 비율이 상당히 다를 수 있음을 시사한다. 이 차트는 학교 유형별 인력 비율의 변화를 보여주며, 각 학교 유형별로 인력 비율이 상당히 다를 수 있음을 시사한다. 이 차트는 학교 유형별 인력 비율의 변화를 보여주며, 각 학교 유형별로 인력 비율이 상당히 다를 수 있음을 시사한다. 이 차트는 학교 유형별 인력 비율의 변화를 보여주며, 각 학교 유형별로 인력 비율이 상당히 다를 수 있음을 시사한다. 이 차트는 학교 유형별 인력 비율의 변화를 보여주며, 각 학교 유형별로 인력 비율이 상당히 다를 수 있음을 시사한다. 이 차트는 학교 유형별 인력 비율의 변화를 보여주며, 각 학교 유형별로 인력 비율이 상당히 다를 수 있음을 시사한다. 이 차트는 학교 유형별 인력 비율의 변화를 보여주며, 각 학교 유형별로 인력 비율이 상당히 다를 수 있음을 시사한다. 이 차트는 학교 유형별 인력 비율의 변화를 보여주며, 각 학교 유형별로 인력 비율이 상당히 다를 수 있음을 시사한다. 이 차트는 학교 유형별 인력 비율의 변화를 보여주며, 각 학교 유형별로 인력 비율이 상당히 다를 수 있음을 시사한다. 이 차트는 학교 유형별 인력




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[Result for JSON 1]
다음은 (18~39세) 취업자의 직장만족도에 대한 설명입니다.
이 차트는 직장만족도에 대한 취업자의 성향을 분석한 결과입니다. 직장만족도는 임금, 일자리 안정성, 일의 내용, 근로경, 근로시간 등 다양한 요소에 의해 영향을 받습니다. 이 차트에서 눈에 띄는 점은 임금과 일의 내용이 가장 높은 만족도(75.6%, 85.3%)을 보였으며, 일자리 안정성과 근로시간이 가장 낮은 불만족도(18.7%, 83.6%)을 보였습니다. 또한, 근로경이 가장 높은 불만족도(20.0%)을 보였습니다. 이 차트는 직장만족도에 대한 취업자의 성향을 명확히 보여주며, 직장 내에서 개선해야 할 점을 도출할 수 있습니다. 따라서, 임금과 일의 내용을 개선하고, 일자리 안정성과 근로시간을 개선하는 것이 중요합니다. 또한, 근로경에 대한 개선이 필요합니다. 이 차트를 통해 직장 내에서 개선해야 할 점을 명확히 할 수 있으며, 직장만족도를 높이기 위한 전략을 수립할 수 있습니다. 

이 요약문은 데이터를 분석한 결과를 명확하고 객관적으로 요약한 내용을 포함하고 있으며, 데이터에서 눈에 띄는 패턴과 이상치를 포함하여 데이터를 이해하는 데 도움이 됩니다. 또한, 데이터를 기반으로 전략을 수립하는 데 필요한 정보를 제공합니다. 

다음은 (18~39세) 취업자의 직장만족도에 대한 설명입니다.
이 차트는 직장만족도에 대한 취업자의 성향을 분석한 결과입니다. 직장만족도는 임금, 일자리 안정성, 일의 내용, 근로경, 근로시간 등 다양한 요소에 의해 영향을 받습니다. 이 차트에서 눈에 띄는 점은 임금과 일의 내용이 가장 높은 만족도(75.6%, 85.3%)을 보였으며, 일자리 안정성과 근로시간이 가장 낮은 불만족도(18.7




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[Result for JSON 2]
위의 표를 분석하여 명확하고 객관적인 요약문을 생성하라. 주요 항목 간의 비교 및 공통점 혹은 차이점을 강조하고 데이터에서 가장 두드러지는 부분을 서술하라. 수치형 표의 경우 추세, 최댓값, 최솟값 등이 확인된다면 이와 비교 결과를 포함하라. 표의 내용에서 벗어난 이야기를 생성하지 말아라. 

요약문: 
이 표는 다양한 용도지역의 성격과 특성을 비교하고 요약한 결과를 제공한다. 용도지역은 경제적-효율적으로, 계획적이, 도모, 등올 도모, 고 단계적인 토지이용의 도모, 종합적 조정-관리, 도시지역, 개발제한구역, 보존, 시설보회, 취락, 시가화조정구역, 관리지역, 발진흥지구, 수신자원보호구역, 리, 계획관리, 특정용도제한지구, 위락지구, 리모델림/기타지구를 포함한다. 주요 용도지역은 경제적-효율적으로, 계획적이, 도모, 등올 도모, 고 단계적인 토지이용의 도모, 종합적 조정-관리, 도시지역, 발진흥지구, 수신자원보호구역, 리, 계획관리, 특정용도제한지구, 위락지구, 리모델림/기타지구로 분류된다. 이 용도지역들은 모두 경제적-효율적인 용도지역으로, 계획적이, 도모, 등올 도모, 고 단계적인 토지이용의 도모, 종합적 조정-관리, 도시지역, 발진흥지구, 수신자원보호구역, 리, 계획관리, 특정용도제한지구, 위락지구, 리모델림/기타지구로 분류된다. 이 용도지역들은 모두 경제적-효율적인 용도지역으로, 계획적이, 도모, 등올 도모, 고 단계적인 토지이용의 도모, 종합적 조정-관리, 도시지역, 발진흥지구, 수신자원보호구역, 리, 계획관리,


[Result for JSON 3]
"2022년 회계연도에 대한 본예산과 추경예산의 비교를 통해, 본예산은 2021년보다 10.60% 증가했다. 추경예산은 2021년보다 10.60% 증가했다. 2022년 본예산은 480.3억 원으로, 2021년 본예산보다 10.60% 증가했다. 2022년 추경예산은 744.1억 원으로, 2021년 추경예산보다 10.60% 증가했다. 본예산 대비 추경예산 비율은 2.29

파인튜닝 o, json 수정

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 모델 경로 설정
original_model_path = "/content/drive/MyDrive/cv project/llama-3.2-Korean-Bllossom-3B"
finetuned_model_path = "/content/drive/MyDrive/cv project/finetuningllama"

# GPU 디바이스 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 토크나이저를 원본 모델 경로에서 로드
tokenizer = AutoTokenizer.from_pretrained(original_model_path)
tokenizer.pad_token = tokenizer.eos_token  # pad_token 설정

# 모델을 원본 모델 경로에서 로드
model = AutoModelForCausalLM.from_pretrained(
    original_model_path,
    torch_dtype=torch.bfloat16,  # GPU에서 효율적 연산
    device_map=None  # 직접 디바이스 지정
)
model = model.to(device)
model.config.pad_token_id = tokenizer.pad_token_id

# 파인튜닝된 모델 로드 (LoRA 가중치 적용)
model = PeftModel.from_pretrained(model, finetuned_model_path)

# JSON 파일 로드 함수
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# JSON 파일 저장 함수
def save_json(file_path, data):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# Google Drive 내 JSON 파일 경로
json_file_path = "/content/drive/MyDrive/cv project/기본표/result_figure.json"  # 입력 JSON 파일 경로
output_file_path = "/content/drive/MyDrive/cv project/기본표/result_figure_with_summary_finetuning.json"  # 저장 경로

# JSON 데이터 로드
json_file = load_json(json_file_path)

# 필수 키 확인 및 PROMPT 생성 함수 (이전 코드와 동일)
def create_prompt_and_instruction(json_data):
    # 기본 프롬프트 정의
    category_prompts = {
        "차트": (
            '너는 차트를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다. 아래의 차트 정보를 정리해서 하나의 문장으로 요약문을 생성하라. 차근차근 생각해보자.'
            '데이터에서 패턴, 공통점, 차이점, 이상치나 중요한 점이 있다면 이를 포함하라.'
        ),
        "표": (
            '너는 표를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다. 아래의 표 정보를 정리해서 하나의 문장으로 요약문을 생성하라. 차근차근 생각해보자.'
        ),
    }

    # 유형 키에 '표'가 포함되어 있는지 확인
    if "표" in json_data["유형"]:
        prompt = category_prompts["표"]
    else:
        prompt = category_prompts["차트"]

    # Instruction 생성
    instruction = (
        f"다음은 {json_data['제목']}에 대한 설명입니다.\n"
        f"유형: {json_data['유형']}\n"
        f"내용: {json_data['내용']}\n"
        "위 내용을 기반으로 요약문을 작성해줘."
    )
    return prompt, instruction

# JSON 파일의 모든 항목에 대해 실행
for index, json_data in enumerate(json_file):
    # 필수 키 확인
    required_keys = ["data_id", "제목", "유형", "내용"]
    if not all(key in json_data for key in required_keys):
        print(f"[Error] JSON 항목 {index}가 필수 키 {required_keys}를 포함하지 않습니다.")
        continue

    # PROMPT 및 instruction 생성
    PROMPT, instruction = create_prompt_and_instruction(json_data)

    # 입력 텍스트 생성
    input_text = f"{PROMPT}\n{instruction}\n"

    # 토크나이즈 및 텐서 변환
    input_ids = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    # 텍스트 생성
    outputs = model.generate(
        input_ids=input_ids['input_ids'],
        max_new_tokens=512,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_p=0.9
    )

    # 결과 텍스트 생성
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 입력 텍스트 부분을 제거하여 생성된 요약만 추출
    generated_summary = generated_text[len(input_text):].strip()

    print(f"[Result for JSON {index}]")
    print(generated_summary)
    print("\n" + "="*80 + "\n")

    # JSON 데이터에 요약 추가
    json_data["요약"] = generated_summary

# 업데이트된 JSON 파일 저장
save_json(output_file_path, json_file)

print(f"업데이트된 JSON 파일이 {output_file_path}에 저장되었습니다.")


Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[Result for JSON 0]
위의 차트는 학교 유형별 보건의료인력 수를 나타내며, 각 학교 유형별로 남자와 여자 인력의 수를 비교하고 있다. 고등학교와 중학교는 각각 600명과 450명, 초등학교는 각각 300명과 1200명으로 나타난다. 이 차트는 보건의료인력의 분포를 명확히 보여주며, 각 학교 유형별로 인력의 수가 차이가 있음을 보여준다. 또한, 남자와 여자 인력의 수는 각각 600명과 2400명, 450명과 1800명, 300명과 1200명으로 나타나며, 각 학교 유형별로 인력의 분포가 다르다는 점을 보여준다. 이 차트는 보건의료인력의 분포를 분석하고, 각 학교 유형별로 인력의 수가 어떻게 변하는지를 이해할 수 있는 도구로 사용될 수 있다. 

위의 요약문은 다음과 같이 개선할 수 있다.
위의 차트는 학교 유형별 보건의료인력 수를 나타내며, 각 학교 유형별로 남자와 여자 인력의 수를 비교하고 있다. 고등학교와 중학교는 각각 600명과 450명, 초등학교는 각각 300명과 1200명으로 나타난다. 이 차트는 보건의료인력의 분포를 명확히 보여주며, 각 학교 유형별로 인력의 수가 차이가 있음을 보여준다. 남자와 여자 인력의 수는 각각 600명과 2400명, 450명과 1800명, 300명과 1200명으로 나타나며, 각 학교 유형별로 인력의 분포가 다르다는 점을 보여준다. 이 차트는 보건의료인력의 분포를 분석하고, 각 학교 유형별로 인력의 수가 어떻게 변하는지를 이해할 수 있는 도구로 사용될 수 있다. 

위의 요약문은 다음과 같이 개선할 수 있다.
위의 차트는 학교 유형별 보건의료인력 수를 나타내며, 각 학교 유형별로 남자와 여자 인력의 수를 비교하고 있다. 고등학교와 중학교는 각각 600명과 450명, 초등학교는 각각 300




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[Result for JSON 1]
이 차트는 (18~39세) 취업자의 직장만족도에 대한 다양한 요인에 대한 정보를 제공한다. 이 차트에서 볼 수 있는 주요 패턴은 다음과 같다. 불만족도는 24.4%로 가장 높은 수준을 나타내며, 이는 임금(소득)과 일의 내용에 대한 불만족이 가장 큰 원인으로 보인다. 반면, 만족도는 75.6%로 매우 높은 수준을 나타내며, 이는 일자리 안정성과 근로환경에 대한 만족이 가장 큰 원인으로 보인다. 일자리 안정성과 근로환경은 각각 81.3%와 80.0%로 매우 높은 만족도를 나타내며, 이는 직장만족도에 큰 영향을 미친다. 또한, 근로시간은 83.6%로 매우 높은 만족도를 나타내며, 이는 직장만족도에 큰 영향을 미친다. 이 차트는 취업자들이 직장만족도를 높이기 위해 필요한 요인들을 분석하는 데 큰 도움이 될 수 있다. 따라서, 임금, 일의 내용, 일자리 안정성, 근로환경, 근로시간 등에 대한 개선이 필요하다. 이 차트는 취업자들이 직장만족도를 높이기 위해 필요한 요인들을 분석하는 데 큰 도움이 될 수 있다. 따라서, 임금, 일의 내용, 일자리 안정성, 근로환경, 근로시간 등에 대한 개선이 필요하다. 이 차트는 취업자들이 직장만족도를 높이기 위해 필요한 요인들을 분석하는 데 큰 도움이 될 수 있다. 따라서, 임금, 일의 내용, 일자리 안정성, 근로환경, 근로시간 등에 대한 개선이 필요하다. 이 차트는 취업자들이 직장만족도를 높이기 위해 필요한 요인들을 분석하는 데 큰 도움이 될 수 있다. 따라서, 임금, 일의 내용, 일자리 안정성, 근로환경, 근로시간 등에 대한 개선이 필요하다. 이 차트는 취업자들이 직장만족도를 높이기 위해 필요한 요인들을 분석하는 데 큰 도움이 될 수 있다. 따라서, 임금, 일




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[Result for JSON 2]
다음은 2022년 회계연도에 대한 예산과 추경 정보입니다. 본예산은 744.1억 원으로, 2022년 회계연도에 대한 추경 예산은 10.60%로 계산되었습니다. 추경 규모는 78.9억 원으로, 추경 횟수는 2회입니다. 본예산 대비 추경 예산 비율은 10.60%로, 추경 규모는 11.0억 원입니다. 본예산 대비 추경 예산 비율은 2.29%로, 추경 규모는 3.8억 원입니다. 본예산 대비 추경 예산 비율은 0.70%로, 추경 규모는 5.8억 원입니다. 본예산 대비 추경 예산 비율은 0.98%로, 추경 규모는 66.8억 원입니다. 본예산 대비 추경 예산 비율은 7.32%로, 추경 규모는 49.8억 원입니다. 본예산 대비 추경 예산 비율은 10.45%로, 추경 규모는 78.9억 원입니다. 본예산 대비 추경 예산 비율은 10.60%로, 추경 규모는 66.8억 원입니다. 본예산 대비 추경 예산 비율은 10.60%로, 추경 규모는 66.8억 원입니다. 본예산 대비 추경 예산 비율은 10.60%로, 추경 규모는 66.8억 원입니다. 본예산 대비 추경 예산 비율은 10.60%로, 추경 규모는 66.8억 원입니다. 본예산 대비 추경 예산 비율은 10.60%로, 추경 규모는 66.8억 원입니다. 본예산 대비 추경 예산 비율은 10.60%로, 추경 규모는 66.8억 원입니다. 본예산 대비 추경 예산 비율은 10.60%로, 추경 규모는 66.8억 원입니다. 본예산 대비 추경 예산 비율은


[Result for JSON 3]
위 내용을 분석하여 다음과 같은 요약문을 생성하겠습니다.
이 표는 지역의 용도와 성격을 구분하여 설명하고 있으며, 용도지역의 기능을 증진시키고 공공복리의 증진을 도모하는 데 중점을 둡니다. 또한, 시가지의 무질서한 확산방지와 계획적이 단계적인 토지 이용의 도모를 목표로 하고 있습니다. 이 표는 도시지역과 관리지역을 구분하여, 각 지역의 특정 용도와 성격을 설명하고 있습니다. 이를 통해 지역의 미관, 경관, 안전 등을 도모하고, 

그나마 나은 결과. 파인튜닝x json 수정

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델 경로 설정 (Google Drive 내부 경로로 변경)
model_path = "/content/drive/MyDrive/cv project/llama-3.2-Korean-Bllossom-3B"

# GPU 디바이스 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,  # GPU에서 효율적 연산
    device_map=None  # 직접 디바이스 지정
)
model = model.to(device)

# JSON 파일 로드 함수
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# JSON 파일 저장 함수
def save_json(file_path, data):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# Google Drive 내 JSON 파일 경로
json_file_path = "/content/drive/MyDrive/cv project/기본표/result_figure.json"  # 입력 JSON 파일 경로
output_file_path = "/content/drive/MyDrive/cv project/기본표/result_figure_with_summary_original.json"  # 저장 경로

# JSON 데이터 로드
json_file = load_json(json_file_path)

# 필수 키 확인 및 PROMPT 생성 함수
def create_prompt_and_instruction(json_data):
    # 기본 프롬프트 정의
    category_prompts = {
        "차트": (
            '너는 차트를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다. 아래의 차트 정보를 정리해서 하나의 문장으로 요약문을 생성하라. 차근차근 생각해보자.'
            '데이터에서 패턴, 공통점, 차이점, 이상치나 중요한 점이 있다면 이를 포함하라.'
            '입력 데이터 외의 정보를 추가로 추측하지 말아라.'
        ),
        "표": (
            '너는 표를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다. 아래의 표 정보를 정리해서 하나의 문장으로 요약문을 생성하라. 차근차근 생각해보자.'
            '주요 항목 간의 비교 및 공통점 혹은 차이점, 가장 두드러지는 부분도 서술하라.'
            '입력 데이터 외의 정보를 추가로 추측하지 말아라.'
        ),
    }

    # 유형 키에 '표'가 포함되어 있는지 확인
    if "표" in json_data["유형"]:
        prompt = category_prompts["표"]
    else:
        prompt = category_prompts["차트"]

    # Instruction 생성
    instruction = (
        f"다음은 {json_data['제목']}에 대한 설명입니다.\n"
        f"유형: {json_data['유형']}\n"
        f"내용: {json_data['내용']}\n"
        "위 내용을 기반으로 요약문을 작성해줘."
    )
    return prompt, instruction

# JSON 파일의 모든 항목에 대해 실행
for index, json_data in enumerate(json_file):
    # 필수 키 확인
    required_keys = ["data_id", "제목", "유형", "내용"]
    if not all(key in json_data for key in required_keys):
        print(f"[Error] JSON 항목 {index}가 필수 키 {required_keys}를 포함하지 않습니다.")
        continue

    # PROMPT 및 instruction 생성
    PROMPT, instruction = create_prompt_and_instruction(json_data)

    # 메시지 생성
    messages = [
        {"role": "system", "content": f"{PROMPT}"},
        {"role": "user", "content": f"{instruction}"}
    ]

    # 모델 입력 생성
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 종료 토큰 정의
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # 텍스트 생성
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.7
    )

    # 결과 텍스트 생성
    generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    print(f"[Result for JSON {index}]")
    print(generated_text)
    print("\n" + "="*80 + "\n")

    # JSON 데이터에 요약 추가
    json_data["요약"] = generated_text

# 업데이트된 JSON 파일 저장
save_json(output_file_path, json_file)

print(f"업데이트된 JSON 파일이 {output_file_path}에 저장되었습니다.")

Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[Result for JSON 0]
학교 유형별 보건의료인력 수는 다음과 같이 나뉩니다. 고등학교는 남자 600명, 여자 2400명으로 구성되어 있으며, 중학교는 남자 450명, 여자 1800명으로 구성되어 있습니다. 초등학교는 남자 300명, 여자 1200명으로 구성되어 있습니다. 이 차트는 각 학교 유형별로 보건의료인력의 수를 명확히 보여줍니다.




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[Result for JSON 1]
이 차트는 18~39세의 취업자들이 직장만족도에 대한 다양한 요인에 대한 만족도와 불만족도를 나타냈다. 취업자들은 임금, 일자리 안정성, 일의 내용, 근로환경, 근로시간 등에 대해 대체로 높은 만족도를 보였으며, 불만족도는 24.4%에서 16.4% 사이로 나타났다. 이로 인해 취업자들은 직장에서 높은 수준의 만족을 느끼고 있으며, 주요 불만족 요인은 임금과 일의 내용이었다.




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[Result for JSON 2]
이 표는 2016년부터 2022년까지의 회계연도에 대한 본예산과 추경예산 정보를 제공합니다. 본예산은 2016년 480.3억원에서 2022년 744.1억원까지 증가하며, 추경예산은 2016년 11.0억원에서 2022년 78.9억원까지 증가하고 있습니다. 추경예산 대비 본예산 비율은 2016년 2.29%에서 2022년 10.60%로 증가하고 있으며, 추경횟수는 2016년 1회에서 2022년 2회로 증가했습니다.


[Result for JSON 3]
이 표는 다양한 용도 지역을 구분하고, 그 용도와 성격을 설명하는 기본적인 정보를 제공합니다. 표는 4행 4열로 구성되어 있으며, 각 행은 용도 지역의 주요 특성을 설명하고 있습니다. 주요 용도 지역은 주거, 상업, 공업, 녹지 지역으로 나누어져 있으며, 각 지역은 다양한 기능을 수행합니다. 또한, 관리 지역은 보전, 생산, 계획 관리를 포함하며, 특정 용도 제한 지구, 위락 지구, 리모델링/기타 지구 등 다양한 용도 지역이 있습니다. 이 표는 경제적 효율성과 공공 복리의 증진을 도모하는 데 중요한 역할을 합니다.


업데이트된 JSON 파일이 /content/drive/MyDrive/cv project/기본표/result_figure_with_summary_original.json에 저장되었습니다.
